In [67]:
import numpy as np
import cv2
import glob
import os

In [68]:
def display(info, img):
    cv2.imshow(info,img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [69]:
img=cv2.imread('sample4.jpg',0)
display('sample1',img)

In [84]:
thresh,img1=cv2.threshold(img,50,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
display('threshold image',img1)

In [71]:
np.array(img).shape[1]

585

In [72]:
# Defining a kernel length
kernel_length = np.array(img).shape[1]//60
print(kernel_length)
# A verticle kernel of (1 X kernel_length), which will detect all the verticle lines from the image.
verticle_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, kernel_length))
# A horizontal kernel of (kernel_length X 1), which will help to detect all the horizontal line from the image.
hori_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (kernel_length, 1))
# A kernel of (3 X 3) ones.
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))

9


In [73]:
# Morphological operation to detect vertical lines from an image
img_temp1 = cv2.erode(img1, verticle_kernel, iterations=3)
cv2.imshow('erosion vertical',img_temp1)
cv2.waitKey(0) 
verticle_lines_img = cv2.dilate(img_temp1, verticle_kernel, iterations=3)
#cv2.imwrite("verticle_lines.jpg",verticle_lines_img)
cv2.imshow('dilation vertical',verticle_lines_img)
cv2.waitKey(0) 
# Morphological operation to detect horizontal lines from an image
img_temp2 = cv2.erode(img1, hori_kernel, iterations=3)
cv2.imshow('erosion horizontal',img_temp2)
cv2.waitKey(0) 
horizontal_lines_img = cv2.dilate(img_temp2, hori_kernel, iterations=3)
#cv2.imwrite("horizontal_lines.jpg",horizontal_lines_img)
cv2.imshow('dilation horizontal',horizontal_lines_img)
cv2.waitKey(0) 
cv2.destroyAllWindows()

In [74]:
# Weighting parameters, this will decide the quantity of an image to be added to make a new image.
alpha = 0.5
beta = 1.0 - alpha
# This function helps to add two image with specific weight parameter to get a third image as summation of two image.
img_final_bin = cv2.addWeighted(verticle_lines_img, alpha, horizontal_lines_img, beta, 0.0)
cv2.imshow('add weight image',img_final_bin)
cv2.waitKey(0) 
cv2.destroyAllWindows()
img_final_bin = cv2.erode(~img_final_bin, kernel, iterations=2)
cv2.imshow('erode image',img_final_bin)
cv2.waitKey(0) 
cv2.destroyAllWindows()
(thresh, img_final_bin) = cv2.threshold(img_final_bin, 128,255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
#cv2.imwrite("img_final_bin.jpg",img_final_bin)Thank
cv2.imshow('threshold image',img_final_bin)
cv2.waitKey(0) 
cv2.destroyAllWindows()

In [85]:
display("final/_bin" , ~img_final_bin)

In [76]:
# Find contours for image, which will detect all the boxes
contours, hierarchy = cv2.findContours(img_final_bin, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
# cv2.imshow('parts',im2)
# cv2.waitKey(0)
# cv2.destroyAllWindows()
    
# Sort all the contours by top to bottom.
#(contours, boundingBoxes) = sort_contours(contours, method="right-to-left")

In [77]:
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)        

In [78]:
contours[0]

array([[[  8, 959]],

       [[  8, 960]],

       [[ 16, 960]],

       [[ 16, 959]]], dtype=int32)

In [79]:
idx = 0
qNo = 10
thresh = 4
x, prevY, w, h = cv2.boundingRect(contours[0])
count = 1
#create folder of questions
for i in range(1, 11):
    createFolder('./Question'+str(i))   
        
name = {}
name[count] = x               
        
for c in contours:
    if qNo > 0:
        # Returns the location and width,height for every contour
        x, y, w, h = cv2.boundingRect(c)
        print("x,y,w,h",x,y,w,h)
        
        if y < prevY + 5 and y > prevY - 5 :
            count = count + 1
            new_img = img[y:y+h, x:x+w]
            cv2.imwrite('Question'+str(qNo)+'/Question'+str(count)+ '.png', new_img)
            name[count] = x
        else :
            count = 1
            if makeFolder == 1:
                qNo = qNo - 1

                rename = sorted(name.items(), key = operator.itemgetter(1), reverse = False)
                print(rename)
                list = ['QuesNo.png', 'a.png', 'b.png', 'c.png', 'd.png', 'e.png', 'marks.png']
                index = 0
                for temp in rename:
                    value, key = temp
                    os.rename("Question"+str(qNo+1)+"/Question"+str(value)+'.png', "Question"+str(qNo+1)+'/'+list[index])
                    index = index + 1
 
                name.clear()
                new_img = img[y:y+h, x:x+w]
                cv2.imwrite('Question'+str(qNo)+'/Question'+str(count)+ '.png', new_img)
                name[count] = x
            else :
                files = glob.glob('Question'+str(qNo)+'/*')
                for f in files:
                    os.remove(f)
                name.clear()    
                new_img = img[y:y+h, x:x+w]
                cv2.imwrite('Question'+str(qNo)+'/Question'+str(count) +'.png', new_img)  
                name[count] = x
        
        if count > thresh :
            makeFolder = 1
        else :
            makeFolder = 0

        prevY = y        

x,y,w,h 8 959 9 2
x,y,w,h 8 950 9 2
x,y,w,h 12 935 573 105
x,y,w,h 49 1008 46 13
x,y,w,h 6 926 4 4
x,y,w,h 0 926 1 1
x,y,w,h 6 924 579 17
x,y,w,h 6 916 1 1
x,y,w,h 0 916 1 1
x,y,w,h 0 909 1 1
x,y,w,h 0 890 1 2
x,y,w,h 18 874 531 56
x,y,w,h 21 893 48 33
x,y,w,h 0 866 6 17
x,y,w,h 5 834 5 21
x,y,w,h 17 822 530 56
x,y,w,h 0 821 10 7
x,y,w,h 17 770 529 54
x,y,w,h 18 719 425 52
x,y,w,h 449 718 95 45
x,y,w,h 19 673 67 45
x,y,w,h 92 671 66 47
x,y,w,h 164 670 66 46
x,y,w,h 236 669 66 46
x,y,w,h 377 668 64 44
x,y,w,h 307 668 64 45
x,y,w,h 448 667 94 44
x,y,w,h 20 621 66 45
[(1, 19), (2, 92), (3, 164), (4, 236), (6, 307), (5, 377), (7, 448)]
x,y,w,h 93 620 65 46
x,y,w,h 236 619 65 45
x,y,w,h 165 619 65 46
x,y,w,h 307 618 64 45
x,y,w,h 447 617 93 43
x,y,w,h 377 617 63 44
x,y,w,h 21 571 66 44
[(1, 20), (2, 93), (4, 165), (3, 236), (5, 307), (7, 377), (6, 447)]
x,y,w,h 93 570 65 45
x,y,w,h 236 569 65 44
x,y,w,h 165 569 65 45
x,y,w,h 376 568 64 44
x,y,w,h 307 568 64 45
x,y,w,h 446 567 93 44
x,y,w,h 

In [80]:
type(name)

dict

In [81]:
print(contours)

[array([[[  8, 959]],

       [[  8, 960]],

       [[ 16, 960]],

       [[ 16, 959]]], dtype=int32), array([[[  8, 950]],

       [[  8, 951]],

       [[ 16, 951]],

       [[ 16, 950]]], dtype=int32), array([[[ 578,  935]],

       [[ 577,  936]],

       [[ 554,  936]],

       [[ 553,  937]],

       [[ 540,  937]],

       [[ 539,  938]],

       [[ 516,  938]],

       [[ 515,  939]],

       [[ 498,  939]],

       [[ 497,  940]],

       [[ 466,  940]],

       [[ 465,  941]],

       [[ 346,  941]],

       [[ 345,  942]],

       [[ 276,  942]],

       [[ 275,  943]],

       [[ 234,  943]],

       [[ 233,  944]],

       [[ 211,  944]],

       [[ 210,  945]],

       [[ 181,  945]],

       [[ 180,  946]],

       [[ 160,  946]],

       [[ 159,  947]],

       [[ 108,  947]],

       [[ 107,  948]],

       [[  58,  948]],

       [[  57,  949]],

       [[  28,  949]],

       [[  27,  950]],

       [[  22,  950]],

       [[  22,  951]],

       [[  25,  951]],

   

In [82]:
print(boundingBoxes)

((547, 525, 11, 150), (500, 150, 31, 7), (477, 29, 7, 32), (449, 718, 95, 45), (448, 667, 94, 44), (447, 617, 93, 43), (446, 567, 93, 44), (445, 518, 93, 44), (444, 469, 93, 43), (444, 420, 92, 43), (444, 371, 92, 44), (444, 322, 91, 44), (444, 272, 91, 45), (444, 223, 90, 45), (443, 172, 91, 47), (411, 37, 64, 35), (377, 668, 64, 44), (377, 617, 63, 44), (376, 568, 64, 44), (376, 518, 63, 44), (375, 469, 63, 44), (375, 420, 63, 43), (375, 371, 63, 43), (375, 321, 63, 44), (375, 271, 63, 44), (375, 222, 63, 44), (374, 171, 64, 45), (342, 29, 24, 34), (313, 29, 24, 33), (307, 668, 64, 45), (307, 618, 64, 45), (307, 568, 64, 45), (306, 519, 64, 44), (306, 469, 64, 44), (305, 420, 64, 43), (305, 370, 64, 44), (305, 321, 64, 44), (305, 271, 64, 44), (305, 222, 64, 43), (304, 171, 64, 45), (285, 29, 23, 33), (256, 29, 23, 33), (249, 22, 204, 47), (236, 669, 66, 46), (236, 619, 65, 45), (236, 569, 65, 44), (236, 519, 64, 44), (235, 469, 65, 44), (235, 420, 65, 43), (235, 370, 64, 44), (235, 

In [83]:
# # import the necessary packages
# import numpy as np
# import argparse
# import imutils
# import cv2

# def sort_contours(cnts, method="left-to-right"):
# 	# initialize the reverse flag and sort index
# 	reverse = False
# 	i = 0

# 	# handle if we need to sort in reverse
# 	if method == "right-to-left" or method == "bottom-to-top":
# 		reverse = True

# 	# handle if we are sorting against the y-coordinate rather than
# 	# the x-coordinate of the bounding box
# 	if method == "top-to-bottom" or method == "bottom-to-top":
# 		i = 1

# 	# construct the list of bounding boxes and sort them from top to
# 	# bottom
# 	boundingBoxes = [cv2.boundingRect(c) for c in cnts]
# 	(cnts, boundingBoxes) = zip(*sorted(zip(cnts, boundingBoxes),
# 		key=lambda b:b[1][i], reverse=reverse))

# 	# return the list of sorted contours and bounding boxes
# 	return (cnts, boundingBoxes)